In [ ]:
def WS_LS(Bandit, epsilon):
    n_trial = np.shape(Bandit)[1]
    trial_number = 0
    First_element = np.random.randint(0, 2,1)[0]
    response  = [First_element]
    reward = [Bandit[response[-1], trial_number]]
    for i in range(1, n_trial):
        if np.random.uniform(low=0.0, high=1.0, size=1)[0]>epsilon:
            if reward[-1] > 0:
                response.append(response[-1])
            else:
                response.append(np.abs(response[-1]-1))
        else :
            response.append(np.random.randint(0, 2,1)[0])
        reward.append(Bandit[response[-1], i])
    return(response, reward)



def RW_softmax(Bandit, alpha, beta):
    # Initialize
    n_trial = np.shape(Bandit)[1]
    trial_number = 0
    First_element = np.random.randint(0, 2,1)[0]
    response  = [First_element]
    reward = [Bandit[response[-1], trial_number]]
    if First_element==0:
        Q0 = [0 + alpha*(reward[-1] - 0)]
        Q1 = [0]
    elif First_element==1:
        Q1 = [0 + alpha*(reward[-1] - 0)]
        Q0 = [0]
    p0 = [np.exp(beta*Q0[-1])/(np.exp(beta*Q0[-1]) + np.exp(beta*Q1[-1]))]

    # Run for each trial
    for i in range(1,n_trial):
        if np.random.uniform(low=0.0, high=1.0, size=1)[0] < p0[-1]:
            response.append(0)
        else:
            response.append(1)
        # Reward and update the Q and p values
        reward.append(Bandit[response[-1], i])
        if response[-1] == 0:
            Q0.append(Q0[-1] + alpha * (reward[-1] - Q0[-1]))
            Q1.append(Q1[-1])
        elif response[-1] == 1:
            Q1.append(Q1[-1] + alpha * (reward[-1] - Q1[-1]))
            Q0.append(Q0[-1])
        p0.append(np.exp(beta * Q0[-1]) / (np.exp(beta * Q0[-1]) + np.exp(beta * Q1[-1])))
    return (response, reward, Q0, Q1, p0)


In [ ]:
def Estimate_likelihood_WSLS(epsilon, response, reward):
    LL = -np.log(0.5)
    for i in range(1, np.shape(response)[0]):
        if ((reward[i - 1] > 0) & (response[i - 1] == response[i])) | ((reward[i - 1] < 0) & (response[i - 1] != response[i])):
            LL += -np.log((1 - epsilon / 2))
        else:
            LL += -np.log((epsilon / 2))
    return (LL)

def Estimate_likelihood_RW_softmax(params, response, reward):
    alpha, beta = params
    LL = -np.log(0.5)
    Q0 = 0
    Q1 = 0
    for i in range(1, np.shape(response)[0]):
        if response[i-1] == 0:
            Q0 = Q0 + alpha * (reward[i-1] - Q0)
        elif response[i-1] == 1:
            Q1 = Q1 + alpha * (reward[i-1] - Q1)
        p0 = np.exp(beta * Q0) / ((np.exp(beta * Q0) + np.exp(beta * Q1)))
        if (response[i]==0):
            LL +=  -np.log(p0)
        else:
            LL += -np.log(1-p0)
    return (LL)